In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.functions import sum
import datetime

In [2]:
# in this notebook we will process requests in the period of Jule 2018
startDate = '20180701'
endDate = '20180731'

In [3]:
# define format for string representing a date and time
format = '%Y%m%d'

In [4]:
# # build spark session
sess = SparkSession.builder.master("local[*]").appName("mmds2").getOrCreate()

In [5]:
# read csv with Wiki interlanguage link records (file that we received after converting from langlinks.sql.gz)
df_lang = sess.read.format("csv").option("header", "true").load("./data/langlinks/ukwiki-20180701-langlinks.csv")

In [6]:
# let's look - just for check
test = df_lang.filter(df_lang.id == 1273710)

In [7]:
test.show()

+-------+----+-----------------+
|     id|lang|            title|
+-------+----+-----------------+
|1273710|  de|PHP-Beschleuniger|
|1273710|  en|  PHP accelerator|
|1273710|  es|  PHP accelerator|
|1273710|  lv| PHP paātrinātājs|
|1273710|  ru|  Акселератор PHP|
|1273710|  zh|          PHP加速軟件|
+-------+----+-----------------+



In [8]:
# filter to English Wiki
dl = df_lang.filter(df_lang.lang == "en")

In [9]:
# take a look at the schema
dl.printSchema()

root
 |-- id: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- title: string (nullable = true)



In [10]:
# take a look at the data
dl.show()

+-------+----+-----+
|     id|lang|title|
+-------+----+-----+
|      3|  en| null|
|  93215|  en| null|
| 109484|  en| null|
| 122051|  en| null|
| 130506|  en| null|
| 609236|  en| null|
| 621969|  en| null|
| 772924|  en| null|
| 907757|  en| null|
| 942309|  en| null|
|1336102|  en| null|
|1627304|  en| null|
|1631181|  en| null|
|1979525|  en| null|
|1979527|  en| null|
|1979528|  en| null|
|1982832|  en| null|
|1983737|  en| null|
|2002192|  en| null|
|2052178|  en| null|
+-------+----+-----+
only showing top 20 rows



In [11]:
# getting  Base per-page data (file that we received after converting from page.sql.gz)
df_page = sess.read.format("csv").option("header", "true").load("./data/page/ukwiki-20180701-page.csv")

In [12]:
# take a look at the schema
df_page.printSchema()

root
 |-- page_id: string (nullable = true)
 |-- page_namespace: string (nullable = true)
 |-- title: string (nullable = true)
 |-- page_restrictions: string (nullable = true)
 |-- page_counter: string (nullable = true)
 |-- page_is_redirect: string (nullable = true)
 |-- page_is_new: string (nullable = true)
 |-- page_random: string (nullable = true)
 |-- page_touched: string (nullable = true)
 |-- page_links_updated: string (nullable = true)
 |-- page_latest: string (nullable = true)
 |-- page_len: string (nullable = true)
 |-- page_no_title_convert: string (nullable = true)
 |-- page_content_model: string (nullable = true)
 |-- page_lang: string (nullable = true)



In [13]:
# create a datetime objects from a string representing a startdate and enddate of the period
start = datetime.datetime.strptime(startDate, format)
end = datetime.datetime.strptime(endDate, format)

In [14]:
# define duration/step, the difference between two dates for next reading daily pageviews files.
step = datetime.timedelta(days=1)

In [15]:
# iterate over days in a period with step 1 day and read daily pageviews statistics and collect to parquet
while start <= end:

    date_Str = start.strftime(format)

    print("Date: " + date_Str)
    
    # read all daily pageviews (getting hourly "pageviews" for this date -  from 00 to 23)
    rawData = sess.read.option("delimiter", " ").option("inferSchema", "true").csv("./data/pageviews/pageviews-" + date_Str + "*")
    
    # define columns of interest
    cols = ["project", "title", "requests", "hour_vise"]

    # conversion from Spark RDD to Dataframe
    rawDF = rawData.toDF(*cols)

    # filter - only ukrainian project
    rawDF_uk = rawDF.filter(rawDF.project == "uk")

    # left join "pageviews" with Base per-page data on 'title' because we need $"page_id"
    rawDF_uk_with_id = rawDF_uk.join(df_page, "title")
    #rawDF_uk_with_id.printSchema()
    #rawDF_uk_with_id.show()
    
    # leftanti with 'langlinks' (exclude corresponding row) - because we need just article missing in the enwiki
    rawDF_uk_notEn = rawDF_uk_with_id.join(dl, rawDF_uk_with_id.page_id == dl.id, 'left_anti')
    
    # group by "page_id" - getting daily "pageviews" for this specific page
    ds = rawDF_uk_notEn.groupBy("page_id").agg(sum("requests").alias("sum_requests"))

    #ds.printSchema()
    ds.show()
    
    # add column with date
    enDS = ds.select("page_id", "sum_requests").withColumn("date", sf.lit(date_Str))
    
    # append daily pageviews to parquet
    enDS.write.format("parquet").mode('append').save("./pageviews.parquet")

    start += step

Date: 20180701
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1006370|           5|
|1023531|           1|
|1028644|           1|
|1038555|           1|
|1040262|           1|
|1041410|           1|
|1041445|           1|
|1041457|           1|
|1042185|           1|
|1043412|           2|
|1046207|           2|
| 105705|           1|
| 106022|           4|
|1065129|           1|
|1065294|           3|
|1066069|           1|
|1081405|           2|
|1082047|           1|
|1082303|           1|
| 109180|           2|
+-------+------------+
only showing top 20 rows

Date: 20180702
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1006370|          17|
|1008797|           1|
|1023531|           1|
|1028306|           1|
|1037752|           4|
|1038733|           1|
|1043412|           2|
|1046207|           1|
|1055177|           2|
|1057378|           1|
| 105758|           2|
| 106022|           5|
|1065129|           4|
|1065294|           2|
|

Date: 20180715
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1006370|           9|
|1027338|           1|
| 102952|           1|
|1037103|           1|
|1037752|           7|
|1038555|           1|
|1038733|           1|
|1041445|           1|
|1043412|           2|
|1046207|           2|
|1046887|           1|
|1048314|           2|
| 104870|           1|
|1053155|           1|
|1057160|           1|
| 105758|           2|
|1057593|           1|
| 106022|           3|
|1061955|           1|
|1065294|           1|
+-------+------------+
only showing top 20 rows

Date: 20180716
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1006370|          48|
|1016816|           2|
|1021617|           1|
|1028306|           1|
|1033524|           1|
|1037752|           6|
|1040262|           2|
|1041410|           2|
|1043412|           2|
|1046207|           3|
|1055177|           5|
| 105705|           1|
|1057593|           1|
|1059418|           1|
|

Date: 20180729
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1006370|           8|
|1021617|           1|
|1035614|           1|
|1037752|           2|
|1038555|           1|
|1043412|           2|
|1046207|           1|
|1048314|           1|
| 104870|           1|
|1053155|           1|
|1053497|           2|
|1057863|           1|
| 106022|           3|
|1065294|           4|
|1065828|           2|
|1066479|           1|
|1074690|           1|
|1081405|           4|
|1082047|           2|
|1083168|           1|
+-------+------------+
only showing top 20 rows

Date: 20180730
+-------+------------+
|page_id|sum_requests|
+-------+------------+
|1002887|           1|
|1006370|           7|
|1016816|           1|
|1021617|           3|
|1027644|           1|
|1028644|           1|
|1037752|           3|
|1040262|           2|
|1041445|           1|
|1043412|           1|
| 105705|           4|
| 106022|           4|
| 106090|           1|
|1065294|           3|
|